
<br>
===================================================<br>
Explore and visualize region properties with pandas<br>
===================================================<br>
This toy example shows how to compute the size of every labelled region in a<br>
series of 10 images. We use 2D images and then 3D images. The blob-like<br>
regions are generated synthetically. As the volume fraction (i.e., ratio of<br>
pixels or voxels covered by the blobs) increases, the number of blobs<br>
(regions) decreases, and the size (area or volume) of a single region can get<br>
larger and larger. The area (size) values are available in a pandas-compatible<br>
format, which makes for convenient data analysis and visualization.<br>
Besides area, many other region properties are available.<br>


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
from skimage import data, measure

In [ ]:
fractions = np.linspace(0.05, 0.5, 10)

###################################################################<br>
2D images<br>
=========

In [ ]:
images = [data.binary_blobs(volume_fraction=f) for f in fractions]

In [ ]:
labeled_images = [measure.label(image) for image in images]

In [ ]:
properties = ['label', 'area']

In [ ]:
tables = [measure.regionprops_table(image, properties=properties)
          for image in labeled_images]
tables = [pd.DataFrame(table) for table in tables]

In [ ]:
for fraction, table in zip(fractions, tables):
    table['volume fraction'] = fraction

In [ ]:
areas = pd.concat(tables, axis=0)

Create custom grid of subplots

In [ ]:
grid = plt.GridSpec(2, 2)
ax1 = plt.subplot(grid[0, 0])
ax2 = plt.subplot(grid[0, 1])
ax = plt.subplot(grid[1, :])
# Show image with lowest volume fraction
ax1.imshow(images[0], cmap='gray_r')
ax1.set_axis_off()
ax1.set_title(f'fraction {fractions[0]}')
# Show image with highest volume fraction
ax2.imshow(images[-1], cmap='gray_r')
ax2.set_axis_off()
ax2.set_title(f'fraction {fractions[-1]}')
# Plot area vs volume fraction
areas.plot(x='volume fraction', y='area', kind='scatter', ax=ax)
plt.show()

###################################################################<br>
In the scatterplot, many points seem to be overlapping at low area values.<br>
To get a better sense of the distribution, we may want to add some 'jitter'<br>
to the visualization. To this end, we use `stripplot` (from `seaborn`, the<br>
Python library dedicated to statistical data visualization) with argument<br>
`jitter=True`.

In [ ]:
fig, ax = plt.subplots()
sns.stripplot(x='volume fraction', y='area', data=areas, jitter=True,
              ax=ax)
# Fix floating point rendering
ax.set_xticklabels([f'{frac:.2f}' for frac in fractions])
plt.show()

###################################################################<br>
3D images<br>
=========<br>
Doing the same analysis in 3D, we find a much more dramatic behaviour: blobs<br>
coalesce into a single, giant piece as the volume fraction crosses ~0.25.<br>
This corresponds to the `percolation threshold<br>
<https://en.wikipedia.org/wiki/Percolation_threshold>`_ in statistical<br>
physics and graph theory.

In [ ]:
images = [data.binary_blobs(length=128, n_dim=3, volume_fraction=f)
          for f in fractions]

In [ ]:
labeled_images = [measure.label(image) for image in images]

In [ ]:
properties = ['label', 'area']

In [ ]:
tables = [measure.regionprops_table(image, properties=properties)
          for image in labeled_images]
tables = [pd.DataFrame(table) for table in tables]

In [ ]:
for fraction, table in zip(fractions, tables):
    table['volume fraction'] = fraction

In [ ]:
blob_volumes = pd.concat(tables, axis=0)

In [ ]:
fig, ax = plt.subplots()
sns.stripplot(x='volume fraction', y='area', data=blob_volumes, jitter=True,
              ax=ax)
ax.set_ylabel('blob size (3D)')
# Fix floating point rendering
ax.set_xticklabels([f'{frac:.2f}' for frac in fractions])
plt.show()